# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

## Usage

1.  Execute cell below to load selected materials into global `materials_in` variable
2.  Specify parameters (e.g. `MAX_AREA`) below or use the default values
3.  Click "Run" > "Run All" to run all cells
4.  Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to the description [here](TBA).
   We assume that two input materials are either in the bulk form (e.g. Ni crystal) or layered (e.g. graphene).

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


Execute cell below to install necessary packages and load materials into the environment.

> **_IMPORTANT:_** This cell must be executed before the rest of the cells due to asynchronous nature of materials loading.


In [ ]:
from jupyterlite.utils import install_packages, get_data, set_data

await install_packages("create_interface_with_min_strain_zsl.ipynb")

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in")

## Parameters

### The algorithm parameters

The following parameters are used for ZSL


In [ ]:
ZSL_PARAMETERS = {
    "MAX_AREA": 400,  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

### Select Materials

We set which material is considered "substrate" (bottom) and which "layer" (top) using the indexes from the list of materials passed. Additionally, we also set miller indices and thickness of the corresponding slabs.


In [ ]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index in the list of input materials
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1,  # the index in the list of input materials
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

## Interface Parameters

The resulting interface will be set accordingly


In [ ]:
INTERFACE_PARAMETERS = {"DISTANCE_Z": 3.0}  # in Angstrom

## Create interfaces

All possible combinations within the maximum area considered are produced.


In [ ]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen, from_pymatgen
from operator import itemgetter

pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material)


def create_interfaces(settings):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
            termination,
            gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
            film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
            substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
            in_layers=True,
        ):
            interfaces[termination].append(interface)
    return interfaces, terminations


interfaces, terminations = create_interfaces(
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)

for termination in terminations:
    print(f"Found {len(interfaces[terminations[0]])} interfaces for", termination, "termination")

## Sort interfaces by strain

We use mean absolute strain and sort the output accordingly.


In [ ]:
strain_modes = {
    "VON_MISES": "von_mises_strain",
    "STRAIN": "strain",
    "MEAN": "mean_abs_strain",
}
strain_mode = strain_modes["MEAN"]

# For each interface make a unitary supercell to wrap elements inside the unit cell
for termination in terminations:
    for i, interface in enumerate(interfaces[termination]):
        interfaces[termination][i] = interface["interface"].make_supercell((1, 1, 1), to_unit_cell=True)

# Sort interfaces by ascending strain and then by ascending number of atoms
sorted_interfaces = sorted(
    interfaces,
    key=lambda x: (itemgetter(strain_mode)(x), x["interface"].num_sites),
)
for termination in terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    print("    strain:", sorted_interfaces[termination][0][strain_mode] * 100, "%")
    print("    number of atoms:", sorted_interfaces[termination][0]["interface"].num_sites)

## Plot the number of atoms vs strain

Adjust the plot parameters as needed.


In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from collections import defaultdict

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",
    "Y_SCALE": "log",
}


def plotly_strain_vs_atoms(strain_mode, sorted_interfaces, settings):
    # Group interfaces by strain and number of atoms
    grouped_interfaces = defaultdict(list)
    for termination, interfaces in sorted_interfaces.items():
        for index, i in enumerate(interfaces):
            key = (i[strain_mode] * 100, i["interface"].num_sites)  # in percentage
            grouped_interfaces[key].append((index, termination))

    x = []
    y = []
    hover_text = []
    for (strain, num_sites), indices_and_terminations in grouped_interfaces.items():
        x.append(strain)
        y.append(num_sites)
        # Only include the interface with the smallest index for each termination
        indices_and_terminations = sorted(indices_and_terminations)
        termination_indices = defaultdict(list)
        for index, termination in indices_and_terminations:
            termination_indices[termination].append(index)
        hover_text.append(
            "<br>-----<br>".join(
                f"Termination: {termination}<br>Index: {min(indices)}<br>Strain: {strain:.2f}%<br>Atoms: {num_sites}"
                for termination, indices in termination_indices.items()
            )
        )

    fig = px.scatter(x=x, y=y, log_x=(settings["X_SCALE"] == "log"), log_y=(settings["Y_SCALE"] == "log"))
    fig.update_traces(hovertemplate=hover_text)
    fig.update_layout(
        xaxis=dict(title="Strain (%)"),
        yaxis=dict(title="Number of atoms"),
        hovermode="closest",
        height=settings["HEIGHT"],
    )
    fig.show()


plotly_strain_vs_atoms(strain_mode, sorted_interfaces, settings=PLOT_SETTINGS)
print("Terminations:", terminations)

## Select interface with desired termination and with minimal strain

Select termination by its index in the terminations list (0 by default) and interface by its index in the sorted array for that termination.


In [ ]:
termination_index = 0
interface_index = 0
selected_interface = from_pymatgen(sorted_interfaces[terminations[termination_index]][interface_index]["interface"])

## Pass to the outside runtime

As below


In [ ]:
output_materials = [selected_interface]
set_data("materials", output_materials)